In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=False,
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)  # randomly shift images vertically (fraction of total height))


In [ ]:
train_generator = datagen.flow_from_directory(
    '../input/face-mask-detection/dataset', 
    subset='training',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

val_generator = datagen.flow_from_directory(
    '../input/face-mask-detection/dataset',
    subset='validation',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
vgg=tensorflow.keras.applications.VGG16(include_top=False, input_shape=(224,224,3))

In [ ]:
resnet=tensorflow.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3))

In [ ]:
inception=tensorflow.keras.applications.InceptionV3(include_top=False, input_shape=(224,224,3))

In [ ]:
'''for layer in vgg.layers:
    layer.trainable=False'''

for layer in resnet.layers:
    layer.trainable=False

'''for layer in inception.layers:
    layer.trainable=False'''

In [ ]:
#vgg.summary()
resnet.summary()
#inception.summary()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
flatten_layer= Flatten()(resnet.output)
final_layer= Dense(2,activation='softmax')(flatten_layer)

In [ ]:
#assemble model
model= Model(inputs=resnet.input, outputs= final_layer)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=15)

In [ ]:
model.evaluate_generator(generator=val_generator,steps=len(val_generator))

In [ ]:
train_generator.class_indices

In [ ]:
import cv2

In [ ]:
def get_mask_nomask(filepath):
    img0=cv2.imread(filepath)
    img0=cv2.resize(img0,(224,224))/255
    class_code=model.predict(img0.reshape(1,224,224,3)).argmax()
    return 'with mask' if class_code==0 else 'without mask'

In [ ]:
get_mask_nomask('../input/face-mask-detection/dataset/with_mask/image_123.png')

In [ ]:
get_mask_nomask('../input/face-mask-detection/dataset/without_mask/image_103.png')

In [ ]:
get_mask_nomask('../input/face-mask-detection/dataset/without_mask/image_174.png')

In [ ]:
get_mask_nomask('../input/myownimages/IMG-20210417-WA0005.jpg')

In [ ]:
get_mask_nomask('../input/myownimages/IMG-20210417-WA0006.jpg')

In [ ]:
get_mask_nomask('../input/myownimages/IMG20210308152528.jpg')

In [ ]:
get_mask_nomask('../input/myimage/IMG20200915124631.jpg')

In [ ]:
get_mask_nomask('../input/myimage/IMG20210123131749.jpg')